In [4]:
import transformers
from transformers import AutoTokenizer
import torch
import os
print(f"number of GPUs: torch.cuda.device_count()")
print(torch.__version__)

number of GPUs: torch.cuda.device_count()
2.6.0+rocm6.1


In [5]:
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct" 

if  torch.cuda.is_available():
    device = "cuda"
else:
    raise ValueError("No se reconoció GPU.")

pipeline = transformers.pipeline(
	"text-generation", 
	model=model_id,
	model_kwargs={"torch_dtype": torch.bfloat16},
	device=device
)

# Tokenizer necesario para contar tokens
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda


In [ ]:
# Directorios de entrada y salida
input_dir = "test/"
output_dir = "out/"
os.makedirs(output_dir, exist_ok=True)

# Instrucciones para el modelo
prompt = [
    {"role": "system", 
     "content": 
        """ 
        Quiero que identifiques entidades nombradas que requieren ser anonimizadas en el informe clínico que copio entre comillas al final de esta instrucción. Quiero que me des el resultado en formato .xml in-line, donde las entidades sean identificadas por etiquetas en el mismo texto. Quiero que etiquetes con los criterios MEDDOCAN. A continuación, te muestro un ejemplo que contiene:
        - El texto original del informe en formato plano (.txt)
        - La representación estructurada del mismo en XML con etiquetas semánticas detalladas y posiciones de texto (atributos start, end, text, TYPE, etc.).
        Tu tarea será generar un XML con las mismas reglas de estructura y etiquetado a partir de cada texto clínico. Instrucciones:
        - Conserva el formato exacto del XML del ejemplo.
        - Cada etiqueta tiene que tener el tipo de entidad (`TYPE`) del inventario de MEDDOCAN. Los tipos de entidad que puedes usar son los siguientes: 
            NOMBRE_SUJETO_ASISTENCIA
            EDAD_SUJETO_ASISTENCIA
            SEXO_SUJETO_ASISTENCIA
            FAMILIARES_SUJETO_ASISTENCIA
            NOMBRE_PERSONAL_SANITARIO
            FECHAS
            PROFESION
            HOSPITAL
            CENTRO_SALUD
            INSTITUCION
            CALLE
            TERRITORIO
            PAIS
            NUMERO_TELEFONO
            NUMERO_FAX
            CORREO_ELECTRONICO
            ID_SUJETO_ASISTENCIA
            ID_CONTACTO_ASISTENCIAL
            ID_ASEGURAMIENTO
            ID_TITULACION_PERSONAL_SANITARIO
            ID_EMPLEO_PERSONAL_SANITARIO
            IDENTIF_VEHICULOS_NRSERIE_PLACAS
            IDENTIF_DISPOSITIVOS_NRSERIE
            DIREC_PROT_INTERNET
            URL_WEB
            IDENTIF_BIOMETRICOS
            OTRO_NUMERO_IDENTIF
            OTROS_SUJETO_ASISTENCIA
          - y un campo de comentario (`comment`) vacío
        Cuando te dé un nuevo texto, responde solo con el XML, sin explicaciones adicionales.
    
        Ejemplo - Informe en formato .txt: 
        Datos del paciente.
        Nombre: Emilia García López
        DNI: 12345678A
        Fecha de nacimiento: 12/02/1968
        Género: Mujer
        Domicilio: Calle de la Princesa 123
        Ciudad: Madrid
        Código postal: 28008
        Email: emilia.garcia@ucm.es
        Teléfono fijo: +34 91 123 45 67
        Teléfono móvil: +34 66 789 01 23
        NHC: 1234567
        NASS: 987654321098
        Condición de riesgo: Persona con discapacidad
        Datos asistenciales.
        Médico: Dra. Sofía Pérez García. NC 987654321. Especialista en Medicina de Familia. Hospital Universitario La Paz. Paseo de la Castellana 261. 28046. Madrid. España.
        Fecha de ingreso: 20/03/2022
        Centro de salud: Centro de Salud Villaverde
        Informe clínico del paciente:
        Paciente de 54 años de edad, con antecedentes de enfermedad cardiovascular, que acude al servicio de urgencias con síntomas de dolor torácico radiante al brazo izquierdo y dificultad para respirar. La exploración física revela taquicardia, hipertensión y roncus bilaterales. Los análisis de sangre muestran elevación de la creatinina y la troponina. Se solicitan estudios de imagen (Rx tórax y ecocardiograma) que revelan un infarto de miocardio en el segmento anterior del ventrículo izquierdo. Se instaura tratamiento con anticoagulantes y analgésicos, y se realiza una angioplastia coronaria para restaurar la circulación sanguínea en el área afectada. El paciente se encuentra estable y se le realiza una vigilancia estrecha en la unidad de cuidados intensivos.
        Antecedentes:
        - Enfermedad cardiovascular
        - Hipertensión
        - Diabetes mellitus tipo 2
        Evolución y comentarios:
        El paciente se encuentra en una evolución estable y se le realiza una vigilancia estrecha en la unidad de cuidados intensivos. Se le han realizado análisis de sangre y estudios de imagen para confirmar el diagnóstico de infarto de miocardio y se ha instaurado tratamiento con anticoagulantes y analgésicos. Se le realizará una angioplastia coronaria para restaurar la circulación sanguínea en el área afectada.
        Remitido por:
        Se reenvía a la unidad de cardiología para seguir el tratamiento y la evolución del paciente.
        
        Ejemplo - Informe en formato .xml: lo que debes generar
        <?xml version='1.0' encoding='UTF-8'?>
        <MEDDOCAN>
          <TEXT>
            Datos del paciente.
            Nombre: <TAG TYPE="NOMBRE_SUJETO_ASISTENCIA">Emilia</TAG> <TAG TYPE="NOMBRE_SUJETO_ASISTENCIA">García López</TAG>.
            DNI: <TAG TYPE="ID_SUJETO_ASISTENCIA">12345678A</TAG>.
            Fecha de nacimiento: <TAG TYPE="FECHAS">12/02/1968</TAG>.
            Género: <TAG TYPE="SEXO_SUJETO_ASISTENCIA">Mujer</TAG>.
            Domicilio: <TAG TYPE="CALLE">Calle de la Princesa 123</TAG>.
            Ciudad: <TAG TYPE="TERRITORIO">Madrid</TAG>.
            Código postal: <TAG TYPE="TERRITORIO">28008</TAG>.
            Email: <TAG TYPE="CORREO_ELECTRONICO">emilia.garcia@ucm.es</TAG>.
            Teléfono fijo: <TAG TYPE="NUMERO_TELEFONO">+34 91 123 45 67</TAG>.
            Teléfono móvil: <TAG TYPE="NUMERO_TELEFONO">+34 66 789 01 23</TAG>.
            NHC: <TAG TYPE="ID_SUJETO_ASISTENCIA">1234567</TAG>.
            NASS: <TAG TYPE="ID_ASEGURAMIENTO">987654321098</TAG>.
            Condición de riesgo: <TAG TYPE="OTROS_SUJETO_ASISTENCIA">Persona con discapacidad</TAG>.
            Datos asistenciales.
            Médico: Dra. <TAG TYPE="NOMBRE_PERSONAL_SANITARIO">Sofía</TAG> <TAG TYPE="NOMBRE_PERSONAL_SANITARIO">Pérez García</TAG>. NC <TAG TYPE="ID_TITULACION_PERSONAL_SANITARIO">987654321</TAG>. <TAG TYPE="PROFESION">Especialista en Medicina de Familia</TAG>. <TAG TYPE="HOSPITAL">Hospital Universitario La Paz</TAG>. <TAG TYPE="CALLE">Paseo de la Castellana 261</TAG>. <TAG TYPE="TERRITORIO">28046</TAG>. <TAG TYPE="TERRITORIO">Madrid</TAG>. <TAG TYPE="PAIS">España</TAG>.
            Fecha de ingreso: <TAG TYPE="FECHAS">20/03/2022</TAG>.
            Centro de salud: <TAG TYPE="CENTRO_SALUD">Centro de Salud Villaverde</TAG>.
            Informe clínico del paciente:
            Paciente de <TAG TYPE="EDAD_SUJETO_ASISTENCIA">54 años</TAG> de edad, con antecedentes de <TAG TYPE="OTROS_SUJETO_ASISTENCIA">enfermedad cardiovascular</TAG>, que acude al servicio de urgencias con síntomas de dolor torácico radiante al brazo izquierdo y dificultad para respirar. La exploración física revela taquicardia, hipertensión y roncus bilaterales. Los análisis de sangre muestran elevación de la creatinina y la troponina. Se solicitan estudios de imagen (Rx tórax y ecocardiograma) que revelan un infarto de miocardio en el segmento anterior del ventrículo izquierdo. Se instaura tratamiento con anticoagulantes y analgésicos, y se realiza una angioplastia coronaria para restaurar la circulación sanguínea en el área afectada. El paciente se encuentra estable y se le realiza una vigilancia estrecha en la unidad de cuidados intensivos.
            Antecedentes:
            - <TAG TYPE="OTROS_SUJETO_ASISTENCIA">Enfermedad cardiovascular</TAG>
            - <TAG TYPE="OTROS_SUJETO_ASISTENCIA">Hipertensión</TAG>
            - <TAG TYPE="OTROS_SUJETO_ASISTENCIA">Diabetes mellitus tipo 2</TAG>
            Evolución y comentarios:
            El paciente se encuentra en una evolución estable y se le realiza una vigilancia estrecha en la unidad de cuidados intensivos. Se le han realizado análisis de sangre y estudios de imagen para confirmar el diagnóstico de infarto de miocardio y se ha instaurado tratamiento con anticoagulantes y analgésicos. Se le realizará una angioplastia coronaria para restaurar la circulación sanguínea en el área afectada.
            Remitido por:
            Se reenvía a la unidad de cardiología para seguir el tratamiento y la evolución del paciente.
          </TEXT>
        </MEDDOCAN>
        
        Recordá que en ningún caso debes incluir advertencias, explicaciones ni descripciones sobre la tarea, sobre la instrucción que te he dado o sobre cuestiones de funcionamiento del modelo de lenguaje.
        """},
     ]

# Configuración de tokens
MAX_CONTEXT_TOKENS = 8192
MAX_GENERATION_TOKENS = 4000
MAX_INPUT_TOKENS = MAX_CONTEXT_TOKENS - MAX_GENERATION_TOKENS



# Procesar cada archivo .txt
for filename in os.listdir(input_dir):
    if filename.endswith(".txt"):
        filepath = os.path.join(input_dir, filename)
        with open(filepath, "r", encoding="utf-8") as f:
            texto = f.read()

        # Crear mensaje estilo chat
        prompt_text = prompt[0]["content"]
        messages = [
            {"role": "system", "content": prompt_text},
            {"role": "user", "content": texto}
        ]

        # Calcular tokens de entrada
        full_prompt = prompt_text + texto
        total_tokens = len(tokenizer.encode(full_prompt))
        print(f"{filename}: Tokens de entrada: {total_tokens}")

        # Truncar el prompt si se pasa del límite permitido
        if total_tokens > MAX_INPUT_TOKENS:
            print(f"Truncando prompt: {filename}")
            # Calcular los tokens disponibles para el prompt
            max_tokens_prompt = MAX_INPUT_TOKENS - len(tokenizer.encode(texto))
            
            # Truncar el prompt para ajustarlo al límite de tokens
            prompt_tokens = tokenizer.encode(prompt[0]["content"])
            truncated_prompt_tokens = prompt_tokens[:max_tokens_prompt]
            
            # Decodificar los tokens truncados y actualizar el prompt
            truncated_prompt = tokenizer.decode(truncated_prompt_tokens, skip_special_tokens=True)
            messages[0]["content"] = truncated_prompt

        # Generar texto
        output = pipeline(messages, max_new_tokens=MAX_GENERATION_TOKENS)

        # Extraer solo el contenido generado por el modelo
        respuesta = output[0]["generated_text"][2]["content"]

        # Guardar en .xml
        output_filename = os.path.splitext(filename)[0] + ".xml"
        output_path = os.path.join(output_dir, output_filename)
        with open(output_path, "w", encoding="utf-8") as out_f:
            out_f.write(respuesta)

        print(f"Procesado: {filename} → {output_filename}")

print("Proceso completado.")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


964191583.txt: Tokens de entrada: 2932


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Procesado: 964191583.txt → 964191583.xml
975577618.txt: Tokens de entrada: 2846


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Procesado: 916606884.txt → 916606884.xml
932528658.txt: Tokens de entrada: 2851


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Procesado: 932528658.txt → 932528658.xml
993577188.txt: Tokens de entrada: 3106


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Procesado: 993577188.txt → 993577188.xml
993479328.txt: Tokens de entrada: 3049
Procesado: 993479328.txt → 993479328.xml
Proceso completado.
